In [12]:
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
K.common.set_image_dim_ordering('th')
from keras.utils import np_utils
#from sklearn.cross_validation import StratifiedKFold

In [13]:
PATH = os.getcwd()
print (PATH)

C:\Users\Ujwal\Classification-of-Hyperspectral-Image-master


In [14]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])


In [15]:
# Global Variables
# windowSize = 5
# numPCAcomponents = 30
# testRatio = 0.25

# Load Training Dataset

In [16]:
X_train = np.load("X_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("y_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [17]:
# Reshape into (numberofsamples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))

In [18]:
# convert class labels to on-hot encoding
y_train = np_utils.to_categorical(y_train)

In [19]:
# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

(30, 5, 5)


In [20]:
# number of filters
C1 = 3*numPCAcomponents

In [21]:
# Define the model
model = Sequential()

model.add(Conv2D(C1, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(3*C1, (3, 3), activation='relu'))
model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dense(6*numPCAcomponents, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))

In [22]:
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
29685/29685 [==============================] - 7s 224us/step - loss: 2.4095 - accuracy: 0.2859
Epoch 2/15
29685/29685 [==============================] - 5s 173us/step - loss: 1.5403 - accuracy: 0.5744
Epoch 3/15
29685/29685 [==============================] - 5s 173us/step - loss: 1.0144 - accuracy: 0.6872
Epoch 4/15
29685/29685 [==============================] - 5s 175us/step - loss: 0.7905 - accuracy: 0.7464
Epoch 5/15
29685/29685 [==============================] - 5s 176us/step - loss: 0.6659 - accuracy: 0.7811
Epoch 6/15
29685/29685 [==============================] - 5s 175us/step - loss: 0.5920 - accuracy: 0.8050
Epoch 7/15
29685/29685 [==============================] - 5s 180us/step - loss: 0.5282 - accuracy: 0.8237
Epoch 8/15
29685/29685 [==============================] - 5s 177us/step - loss: 0.4889 - accuracy: 0.8398
Epoch 9/15
29685/29685 [==============================] - 5s 175us/step - loss: 0.4472 - accuracy: 0.8533
Epoch 10/15
29685/29685 [=====================

In [24]:
import h5py
from keras.models import load_model

In [25]:
model.save('my_model' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio) + '.h5')